In [60]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

In [61]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [62]:
def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de problemas
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de usuarios que ha hecho cada problema
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado

In [63]:
def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    

In [64]:
def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo

In [65]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2016-10-21 00:00:00")

print(len(training_set))

# obtengo los nodos del grafo:
nodes = training_set.problem_id.unique()


# creo un diccionario que va a tener a los problemas como keys y los valores seran los
# usuarios que han hecho ese problema
grouped = training_set.groupby('problem_id')['user_id'].apply(list)

print(len(nodes))
# print(grouped)


10262
169


In [66]:
print(training_set)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [67]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2016-10-21 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
16939         469      799     RF  2016-10-21 00:08:23
16940         325     3832     AC  2016-10-21 00:08:35
16942         469     3757     AC  2016-10-21 00:41:36
16943         469     3792    RTE  2016-10-21 01:04:08
16944         469      810     AC  2016-10-21 01:48:35
16945         438     3943     AC  2016-10-21 02:23:56
16946         469     3773     AC  2016-10-21 05:00:58
16947         346     3728     AC  2016-10-21 09:06:38
16948         469     3736     CE  2016-10-21 09:38:14
16949         438     3785     TL  2016-10-21 09:43:52
16950         469     3704     AC  2016-10-21 10:04:03
16951         469     2912     AC  2016-10-21 10:20:36
16952         213     3136     AC  2016-10-21 10:28:23
16953         213     3088     AC  2016-10-21 10:29:26
16954         469     3708    RTE  2016-10-21 10:35:15
16956         235     3136    RTE  2016-10-21 10:37:58
16957         183     3880     AC  2016-10-21 11:38:11
16958     

In [68]:

# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

10837


In [69]:
def apply_wpa(row, graph):
    """
        Funcion que devuelve el valor de weighted preferential attachment
    """
    value_wpa = graph.degree(row["one"], weight="weight") * graph.degree(row["two"], weight="weight")
        
    return value_wpa
    

def create_wpa_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de wpa para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_wpa = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_wpa['wpa'] = dataFrame_wpa.apply (lambda row: apply_wpa(row, graph), axis=1)


    return dataFrame_wpa

In [70]:
wpa_df = create_wpa_data(graph, nodes)
print(wpa_df)

       one  two      wpa
0       10    2  5141996
1       10    4  2736420
2       10    6  2955952
3       10   13  3518696
4       10   15  2878652
5       10   39  7038938
6       10   33  5060058
7       10   44  5435736
8       10   19  1725336
9       10   27  2315908
10      10   60  1643398
11      10  100  3102822
12      10   93   457616
13      10   53  2068548
14      10   51  3098184
15      10   49  4511228
16      10   70  3280612
17      10   62  3560438
18      10   35  2447318
19      10   81  1298640
20      10    8  1646490
21      10   17  1127034
22      10   23  1838194
23      10   25        0
24      10   86  2189136
25      10   29  2393208
26      10   31        0
27      10   47        0
28      10   65  1658858
29      10   73  2020622
...    ...  ...      ...
14166  510  509  2570100
14167  510  507  1277142
14168  511  504        0
14169  511  503  2190100
14170  511  505  2017070
14171  511  506  1431430
14172  511  502  1779910
14173  511  509  1573000


In [71]:
# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user = training_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped_user.index, 'list_problem_id':grouped_user.values})

print(df_users)

                                        list_problem_id  user_id
0                                     [10, 76, 489, 39]        5
1                                              [2, 487]        6
2                      [39, 44, 76, 105, 109, 114, 117]        7
3     [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4          [2, 10, 4, 6, 13, 253, 57, 62, 143, 83, 109]        9
5                                             [307, 39]       10
6                           [33, 109, 39, 93, 100, 155]       11
7     [13, 33, 39, 44, 100, 51, 49, 70, 81, 53, 130,...       12
8                                         [44, 187, 39]       15
9                                                  [39]       16
10                                             [39, 86]       17
11                            [119, 114, 316, 507, 506]       18
12                                                 [44]       22
13                                            [100, 13]       24
14    [258, 141, 309, 310

In [72]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    
    

In [73]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)


      user_id  len_training  len_evaluation
0           5             4               1
1           6             2               0
2           9            11               3
3           8            47               0
4          16             1               0
5          17             2               2
6          12            12               0
7          18             5               8
8          15             3               1
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            41               1
16         33             9               0
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [74]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))


      user_id  len_training  len_evaluation
7          18             5               8
29         60            18              14
118        25            22               7
261       130            80               6
299       414            49              61
316       443            53               8
317       448            43              21
483       689            33               7
619       912            13              11
633       935           127               9
1196     1711           128              16
1288     1893             7               6
1311     1952            19               9
1339     1955             9               5
1416     2038             5               5
1423     2096            12              13
1446     2051            11               8
1447     2025            12               8
1448     2120             7               8
1451     2041             9               7
1528     2257             6              13
1694     2576             5     

In [75]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users)

                                        list_problem_id  user_id
11                            [119, 114, 316, 507, 506]       18
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711
997                      [39, 49, 465, 159, 134, 13, 2]     1893
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952
1024      [325, 327, 441, 468, 471, 503, 226, 109, 446]     1955
1066  [155, 505, 49, 162,

In [76]:
# creo el nuevo dataframe con los resultados 
column_problem_recomend = {'problem_id': nodes}
dataframe_problem_recomend = pd.DataFrame.from_dict(column_problem_recomend)

print(dataframe_problem_recomend)

     problem_id
0            10
1             2
2             4
3             6
4            13
5            15
6            39
7            33
8            44
9            19
10           27
11           60
12          100
13           93
14           53
15           51
16           49
17           70
18           62
19           35
20           81
21            8
22           17
23           23
24           25
25           86
26           29
27           31
28           47
29           65
..          ...
139         444
140         437
141         438
142         443
143         445
144         442
145         440
146         446
147         465
148         471
149         468
150         470
151         469
152         467
153         489
154         485
155         486
156         487
157         484
158         488
159         508
160         510
161         511
162         504
163         503
164         505
165         506
166         502
167         509
168         507

[169 ro

In [77]:
def getSimilarProblems(row, wpa_df):
    """
        Funcion que devuelve la lista de los problemas similares del problema de la fila: aquellos que tienen un valor wpa
        mayor a cero
    """
    # print(row['user_id'])
    
    # obtengo dos df con los usuarios que tienen usuarios vecinos con el usuario de la fila 
    column_result_one_tmp = wpa_df[wpa_df['one'] == row['problem_id']]
    column_result_one = column_result_one_tmp[column_result_one_tmp['wpa'] > 0]
    column_result_two_tmp = wpa_df[wpa_df['two'] == row['problem_id']]
    column_result_two = column_result_two_tmp[column_result_two_tmp['wpa'] > 0]
    
    # saco las listas de usuarios con usuarios comunes
    list_one = list(column_result_one['two'])
    list_two = list(column_result_two['one'])
    
    # la concateno sin tener en cuenta repeticiones, porque nunca va a haber
    list_wpa = list_one + list_two
    
    # print(list_wpa)
    
    if list_wpa == []: # sino tiene vecinos en comun, pongo toda la lista de nodos
        list_wpa = graph.nodes()
        list_wpa.remove(row['problem_id']) # y elimino el nodo que estoy mirando
    
    # hago el filtro de los k mejores
    return list_wpa

In [78]:
dataframe_problem_recomend['neighbors'] = dataframe_problem_recomend.apply (lambda row: getSimilarProblems(row, wpa_df), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_problem_recomend)

     problem_id                                          neighbors
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...
13           93  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 2

In [79]:
print(grouped)
# convierto la serie en un dataframe
df_users_recommend = pd.DataFrame({'problem_id':grouped.index, 'list_user_id':grouped.values})
print(df_users_recommend)

problem_id
2      [6, 9, 33, 67, 56, 84, 35, 60, 112, 124, 106, ...
4      [9, 8, 35, 84, 126, 44, 39, 313, 42, 437, 130,...
6      [9, 46, 35, 67, 89, 84, 137, 152, 231, 221, 88...
8      [35, 52, 133, 129, 130, 437, 511, 605, 935, 67...
10     [5, 9, 35, 133, 129, 80, 137, 247, 437, 103, 5...
13     [9, 12, 59, 35, 67, 109, 54, 24, 55, 106, 136,...
15     [8, 35, 67, 55, 133, 129, 84, 167, 205, 231, 2...
17           [35, 133, 129, 437, 103, 206, 62, 89, 1711]
19     [8, 35, 44, 205, 437, 527, 448, 1063, 206, 62,...
23     [35, 67, 60, 437, 206, 62, 103, 864, 1001, 113...
25                                        [35, 103, 130]
27     [8, 35, 67, 167, 39, 440, 448, 437, 130, 488, ...
29     [35, 133, 86, 112, 42, 232, 248, 247, 314, 320...
31                                        [35, 437, 448]
33     [12, 59, 35, 84, 111, 96, 152, 126, 155, 30, 1...
35     [64, 35, 214, 120, 236, 82, 409, 390, 175, 249...
39     [16, 17, 12, 7, 31, 40, 33, 66, 68, 71, 69, 70...
44     [12, 15, 22, 

In [80]:
def wpa_value(one, two, graph):
    """
        Funcion que devuelve el valor de weighted preferential attachment
    """
    value_wpa = graph.degree(one, weight="weight") * graph.degree(two, weight="weight")
        
    return value_wpa

In [81]:
def getWeights(row, graph):
    """
        Funcion que calcula la suma de todos los valores de wpa de todos los nodos con los que tiene wpa
    """
    
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # print(neighbors_list)
    
    problem = row['problem_id']
    suma = 0
    
    for elem in neighbors_list:
        # print(suma)
        # print(graph[user][elem]['weight'])
        suma = suma + wpa_value(problem, elem, graph)
        
    return suma

In [82]:
def getPonderaciones(row, graph):
    """
        Funcion que calcula la ponderacion para cada problema vecino del de la fila
        Calculo la ponderacion diviendo el peso del enlace que enlaza cada problema con problem_id con la suma 
        total de los pesos
    """
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # obtengo la suma de pesos de los enlaces de esa lista
    total_weight = row['total_weight']
    
    # obtengo el id del usuario al que quiero recomendar
    problem = row['problem_id']
    
    lista_ponderaciones = list()
    
    for elem in neighbors_list:
        
        # obtengo el peso del enlace
        peso_enlace = wpa_value(problem, elem, graph)
        
        if total_weight == 0:
            ponderacion = 0
        else:
            # hago la ponderacion
            ponderacion = peso_enlace/total_weight
        
        lista_ponderaciones.append(ponderacion)
        
    return lista_ponderaciones
    

In [83]:
# ahora voy a incluir una nueva columna que tenga las ponderaciones (suma de todos los enlaces) de cada usuario
dataframe_problem_recomend['total_weight'] = dataframe_problem_recomend.apply (lambda row: getWeights(row, graph), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [84]:
# para cada usuario a "recomendar" para cada problema miro el numero de apariciones en su lista

dataframe_problem_recomend['score'] = dataframe_problem_recomend.apply (lambda row: getPonderaciones(row, graph), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [85]:

# elimino la columna de total_weight, ya que no interesa
del dataframe_problem_recomend['total_weight']

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [86]:
def getListUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de listas la concatenacion de listas de usuarios que han
        hecho los problemas similares a ese. Además eliminara los usuarios que ya hayan hecho el problema
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # lista resultante 
    list_result = list(list())
    
    # obtengo la longitud de la lista de vecinos de ese problema
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de problemas vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de problemas que ha hecho el usuario similar
        list_users_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_users_df['list_user_id'])[0]
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora incluyo la lista en la lista
        list_result.append(list_problems)
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [87]:
dataframe_problem_recomend['lista_users_por_problema'] = dataframe_problem_recomend.apply(lambda row: getListUsersFromSimilarProblems(row, df_users_recommend), axis=1)

print(dataframe_problem_recomend)


     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [88]:
# ahora voy a separar cada problema_user para hacer la cuenta
# creo un nuevo dataframe que agrupa por el problema con su posible usuario
df_separation_neigh = dataframe_problem_recomend.groupby(['problem_id']).neighbors.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation_neigh.columns = ['problem_id', 'neighbors', ]

print(df_separation_neigh)

       problem_id  neighbors
0               2          4
1               2          6
2               2         13
3               2         15
4               2         39
5               2         33
6               2         44
7               2         19
8               2         27
9               2         60
10              2        100
11              2         93
12              2         53
13              2         51
14              2         49
15              2         70
16              2         62
17              2         35
18              2         81
19              2          8
20              2         17
21              2         23
22              2         86
23              2         29
24              2         65
25              2         73
26              2         95
27              2         78
28              2         76
29              2        105
...           ...        ...
27074         511        374
27075         511        379
27076         

In [89]:
# creo un nuevo dataframe que agrupa por el usuario, poniendo en la columna de al lado, el score
df_separation_pond = dataframe_problem_recomend.groupby(['problem_id']).score.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation_pond.columns = ['problem_id', 'score']

print(df_separation_pond)


       problem_id     score
0               2  0.007072
1               2  0.007640
2               2  0.009094
3               2  0.007440
4               2  0.018192
5               2  0.013077
6               2  0.014048
7               2  0.004459
8               2  0.005985
9               2  0.004247
10              2  0.008019
11              2  0.001183
12              2  0.005346
13              2  0.008007
14              2  0.011659
15              2  0.008479
16              2  0.009202
17              2  0.006325
18              2  0.003356
19              2  0.004255
20              2  0.002913
21              2  0.004751
22              2  0.005658
23              2  0.006185
24              2  0.004287
25              2  0.005222
26              2  0.004131
27              2  0.005498
28              2  0.002142
29              2  0.007124
...           ...       ...
27074         511  0.003958
27075         511  0.000166
27076         511  0.001858
27077         511  0

In [90]:
df_separation = df_separation_neigh

df_separation['score'] = df_separation_pond['score']

# creo ahora el nuevo dataframe gracias a los dos df anteriores que eran auxiliares
print(df_separation)


       problem_id  neighbors     score
0               2          4  0.007072
1               2          6  0.007640
2               2         13  0.009094
3               2         15  0.007440
4               2         39  0.018192
5               2         33  0.013077
6               2         44  0.014048
7               2         19  0.004459
8               2         27  0.005985
9               2         60  0.004247
10              2        100  0.008019
11              2         93  0.001183
12              2         53  0.005346
13              2         51  0.008007
14              2         49  0.011659
15              2         70  0.008479
16              2         62  0.009202
17              2         35  0.006325
18              2         81  0.003356
19              2          8  0.004255
20              2         17  0.002913
21              2         23  0.004751
22              2         86  0.005658
23              2         29  0.006185
24              2        

In [91]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada problema
# ahora lo que quiero es ordenar los vecinos por cada problema en funcion de su ponderacion
# primero ordeno por su valor de problem y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
df_separation = df_separation.sort_values(by=['problem_id', 'score'], ascending=False)
print(df_separation)

       problem_id  neighbors     score
26956         511         39  0.018039
26987         511        109  0.014854
26958         511         44  0.013931
26951         511          2  0.013178
26957         511         33  0.012968
27029         511        183  0.012817
27062         511        327  0.012548
27012         511        254  0.012374
27013         511        258  0.011819
27000         511        159  0.011617
26966         511         49  0.011561
26994         511        134  0.011256
27001         511        150  0.011209
27005         511        187  0.011177
26999         511        162  0.011078
26997         511        141  0.011054
27063         511        325  0.010460
27021         511        209  0.010337
26995         511        136  0.010091
27020         511        203  0.010064
27078         511        390  0.009945
26998         511        155  0.009509
27011         511        256  0.009509
27019         511        195  0.009287
27082         511        

In [92]:
# elimino la columna ya que no interesa
del df_separation['score']

print(df_separation)

       problem_id  neighbors
26956         511         39
26987         511        109
26958         511         44
26951         511          2
26957         511         33
27029         511        183
27062         511        327
27012         511        254
27013         511        258
27000         511        159
26966         511         49
26994         511        134
27001         511        150
27005         511        187
26999         511        162
26997         511        141
27063         511        325
27021         511        209
26995         511        136
27020         511        203
27078         511        390
26998         511        155
27011         511        256
27019         511        195
27082         511        441
27007         511        253
26968         511         62
27034         511        191
26954         511         13
26996         511        139
...           ...        ...
149             2        487
18              2         81
39            

In [93]:
# ahora tengo que hacer un nuevo dataframe con problem_id y vecino
# hago primero la agrupacion por problem_id
grouped_r = df_separation.groupby('problem_id')

# hago la agregacion en una lista 
df_aux = grouped_r.aggregate(lambda x:list(x))

print(df_aux)

                                                    neighbors
problem_id                                                   
2           [39, 109, 44, 33, 183, 327, 254, 258, 159, 49,...
4           [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...
6           [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...
8           [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...
10          [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...
13          [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...
15          [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...
17          [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...
19          [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...
23          [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...
25          [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...
27          [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...
29          [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...
31          [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...
33      

In [94]:
#print(sorted(nodes))

# voy a crear un nuevo dataframe con la columna problem_id y neighbors (con el anterior no se puede trabajar sin indices)
df_recommend = pd.DataFrame({'problem_id':sorted(nodes), 'neighbors':df_aux['neighbors'].tolist()})

print(df_recommend)

                                             neighbors  problem_id
0    [39, 109, 44, 33, 183, 327, 254, 258, 159, 49,...           2
1    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...           4
2    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...           6
3    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...           8
4    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          10
5    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          13
6    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          15
7    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          17
8    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          19
9    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          23
10   [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...          25
11   [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          27
12   [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          29
13   [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...        

In [95]:
def getUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de la concatenacion de listas de usuarios que han
        realizado los problemas similares al de la fila. Además eliminara los usuarios que ya hayan realizado el problema.
    """
    
    # obtengo la lista de usuarios que han hecho el problema en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # print(list_problems_user)

    # lista resultante de la concatenacion de las listas de usuarios de los problemas similares
    list_result = list()
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de usuarios que han hecho el problema similar
        list_problems_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_user_id'])[0]
        
        # print("----------")
        #print(lista_problemas_comprobar)
        
        # aqui hago el filtro para que no se incluyan los usuarios que ya ha hechon el problema
        list_users = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora concateno el resultado
        list_result = list_result + list_users
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [96]:
# ahora para cada problema, hacer una lista de los usuarios similares a los que han realizado ese problema, 
# que no sean usuarios que han realizado ya ese problema
df_recommend['list_users'] = dataframe_problem_recomend.apply (lambda row: getUsersFromSimilarProblems(row, df_users_recommend), axis=1)
print(df_recommend)

                                             neighbors  problem_id  \
0    [39, 109, 44, 33, 183, 327, 254, 258, 159, 49,...           2   
1    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...           4   
2    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...           6   
3    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...           8   
4    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          10   
5    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          13   
6    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          15   
7    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          17   
8    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          19   
9    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          23   
10   [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...          25   
11   [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          27   
12   [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          29   
13   [256, 257, 2, 2

In [97]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['list_users'] if not (x in conjunto_vacio or function_add(x))]

In [98]:
# tengo que contar el numero de apariciones de cada usuario en la lista de usuarios

# voy a sacar primero una lista sin repeticiones
# ahora voy a crear una nueva columna que contenga la lista de usuarios sin repeticiones
df_recommend['lista_users_unique'] = df_recommend.apply(lambda row: delRepetitions(row), axis=1)

print(df_recommend)

                                             neighbors  problem_id  \
0    [39, 109, 44, 33, 183, 327, 254, 258, 159, 49,...           2   
1    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...           4   
2    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...           6   
3    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...           8   
4    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          10   
5    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          13   
6    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          15   
7    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          17   
8    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          19   
9    [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          23   
10   [256, 257, 2, 259, 4, 342, 6, 503, 8, 307, 10,...          25   
11   [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          27   
12   [39, 109, 44, 2, 33, 183, 327, 254, 258, 159, ...          29   
13   [256, 257, 2, 2

In [99]:
# ahora voy a separar cada user-problema_a_recomendar para hacer la cuenta
# creo un nuevo dataframe que agrupa por el primer problema y tiene su posible recomendacion
new_df_separation = df_recommend.groupby(['problem_id']).lista_users_unique.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

new_df_separation.columns = ['problem_id', 'possible_user']

print(new_df_separation)

        problem_id  possible_user
0                2              6
1                2             33
2                2             67
3                2             56
4                2             84
5                2             60
6                2            112
7                2            124
8                2            106
9                2             41
10               2            158
11               2            114
12               2            167
13               2            165
14               2            149
15               2            217
16               2            218
17               2            191
18               2             44
19               2            205
20               2            245
21               2            246
22               2            251
23               2            189
24               2            216
25               2            135
26               2            294
27               2            177
28            

In [100]:
def getScoring(row, dataframe_problem_recomend):
    """
        Funcion en la que por cada usuario-problema, hace el scoring para ese problema (para ese usuario)
        sumando todas las ponderaciones de ese problema para ese usuario
    """
     # obtengo la lista de listas de usuarios que han hecho los vecinos del problema
    list_problems_users_df = dataframe_problem_recomend[dataframe_problem_recomend['problem_id'] == row['problem_id']]
    list_user_per_problem = list(list_problems_users_df['lista_users_por_problema'])[0]
    
    # saco tambien la lista de scores por vecinos
    list_scores_per_problem = list(list_problems_users_df['score'])[0]
    
    # print(list_scores_per_user)
    
    # obtengo el usuario del que quiero calcular el scoring
    user = row['possible_user']
    # print(problem)
    
    suma = 0
    k = len(list_user_per_problem)
    
    # por cada lista de la lista, saco la ponderacion de su correspondiente usuario (el usuario que lo ha hecho)
    # si problem aparece en la lista, sumo esa ponderacion
    for i in range(0, k):
        lista_a_comprobar = list_user_per_problem[i]
        # print(lista_a_comprobar)
        if user in lista_a_comprobar:
            # print(list_scores_per_user[i])
            suma = suma + (1/(i+1))
    
    return suma

In [101]:
# ahora voy a calcular el scoring para cada problema-usuario
new_df_separation['score'] = new_df_separation.apply(lambda row: getScoring(row, dataframe_problem_recomend), axis=1)

print(new_df_separation)

        problem_id  possible_user     score
0                2              6  0.000000
1                2             33  0.000000
2                2             67  0.000000
3                2             56  0.000000
4                2             84  0.000000
5                2             60  0.000000
6                2            112  0.000000
7                2            124  0.000000
8                2            106  0.000000
9                2             41  0.000000
10               2            158  0.000000
11               2            114  0.000000
12               2            167  0.000000
13               2            165  0.000000
14               2            149  0.000000
15               2            217  0.000000
16               2            218  0.000000
17               2            191  0.000000
18               2             44  0.000000
19               2            205  0.000000
20               2            245  0.000000
21               2            24

In [102]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada usuario
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su ponderacion
# primero ordeno por su valor de user y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
new_df_separation = new_df_separation.sort_values(by=['possible_user', 'score'], ascending=False)
print(new_df_separation)

        problem_id  possible_user     score
223111         322           4018  1.000000
197646         275           4018  0.500000
201533         282           4018  0.333333
215171         313           4018  0.250000
211223         311           4018  0.200000
213205         312           4018  0.166667
219117         315           4018  0.142857
222411         316           4018  0.125000
217141         314           4018  0.111111
209262         310           4018  0.100000
207328         309           4018  0.090909
205405         308           4018  0.083333
203451         307           4018  0.076923
199594         279           4018  0.071429
102862         145           4018  0.066667
164318         247           4018  0.062500
156680         239           4018  0.058824
168195         250           4018  0.055556
160348         243           4018  0.052632
148765         231           4018  0.050000
150732         233           4018  0.047619
195722         272           401

In [103]:
# ahora tengo que hacer un nuevo dataframe con usuario y problema, y una lista de recommendation 

# hago primero la agrupacion por usuario
grouped_r = new_df_separation.groupby('possible_user')

# hago la agregacion en una lista 
df_recommend = grouped_r.aggregate(lambda x:list(x))

#print(df_recommend)

# vuelvo a crear la estructura de los datos para poder trabajar con ellos
df_recommend = pd.DataFrame({'user_id':df_recommend.index.values, 'problem_id':df_recommend['problem_id'].tolist()})
print(df_recommend)

                                             problem_id  user_id
0     [507, 467, 78, 509, 469, 13, 95, 502, 470, 506...        5
1     [486, 10, 485, 507, 489, 25, 31, 47, 331, 484,...        6
2     [78, 33, 95, 13, 73, 83, 65, 6, 57, 29, 4, 86,...        7
3     [254, 255, 187, 507, 390, 130, 31, 331, 484, 3...        8
4     [507, 509, 502, 70, 88, 191, 105, 506, 25, 31,...        9
5     [279, 145, 13, 247, 6, 239, 4, 250, 2, 243, 10...       10
6     [60, 13, 88, 27, 6, 4, 19, 90, 2, 139, 44, 10,...       11
7     [6, 93, 60, 4, 171, 27, 2, 507, 150, 509, 502,...       12
8     [33, 147, 13, 6, 130, 4, 2, 171, 10, 507, 150,...       15
9     [13, 6, 4, 2, 10, 507, 509, 502, 506, 505, 503...       16
10    [23, 13, 17, 6, 8, 4, 81, 2, 10, 35, 507, 62, ...       17
11    [505, 509, 314, 122, 503, 109, 502, 310, 124, ...       18
12    [33, 39, 15, 13, 6, 4, 2, 10, 507, 509, 502, 5...       22
13    [6, 60, 4, 27, 2, 19, 507, 44, 509, 502, 33, 5...       24
14    [130, 487, 308, 489

In [104]:
def getProblems(row, df_recommend):
    """
        Funcion que copia los posibles problemas a recomendar para los usuarios a los que tengo que recomendar
    """
    
    # saco la lista de problemas 
    df_lista_problemas_original = df_recommend[df_recommend['user_id'] == row['user_id']]
    lista_problemas_original = list(df_lista_problemas_original['problem_id'])[0]
    
    return lista_problemas_original
    

In [105]:
# df_users # en este dataframe estan los usuarios a los que quiero recomendar

# ahora voy a copiar los problemas para los usuarios a los que tengo que recomendar
df_users['recommendation'] = df_users.apply (lambda row: getProblems(row, df_recommend), axis=1)

print(df_users)

                                        list_problem_id  user_id  \
11                            [119, 114, 316, 507, 506]       18   
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25   
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60   
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130   
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414   
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443   
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448   
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689   
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912   
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935   
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711   
997                      [39, 49, 465, 159, 134, 13, 2]     1893   
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952   
1024      [325, 327, 441, 468, 471, 503, 226, 10

In [106]:
# elimino la columna de los problemas, ya que no interesa
del df_users['list_problem_id']

print(df_users)

      user_id                                     recommendation
11         18  [505, 509, 314, 122, 503, 109, 502, 310, 124, ...
14         25  [130, 487, 308, 489, 247, 388, 155, 315, 465, ...
39         60  [162, 10, 93, 119, 272, 70, 239, 437, 97, 17, ...
90        130  [507, 122, 502, 23, 76, 316, 373, 322, 308, 26...
241       414  [119, 256, 249, 507, 257, 159, 266, 88, 122, 5...
255       443  [487, 308, 314, 373, 279, 322, 393, 489, 465, ...
257       448  [247, 313, 487, 511, 231, 39, 239, 253, 62, 93...
392       689  [88, 6, 509, 76, 65, 206, 203, 83, 365, 502, 7...
512       912  [308, 342, 314, 257, 29, 258, 226, 57, 10, 440...
523       935  [166, 358, 440, 88, 313, 206, 31, 383, 245, 76...
933      1711  [486, 373, 31, 484, 383, 488, 440, 262, 322, 3...
997      1893  [6, 446, 119, 51, 162, 507, 509, 440, 122, 502...
1021     1952  [322, 209, 39, 191, 404, 437, 217, 222, 275, 2...
1024     1955  [440, 322, 465, 404, 217, 88, 511, 442, 275, 3...
1066     2025  [322, 33, 

In [107]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    return row['recommendation'][:k]

In [108]:
k = 10

#print(list(range(1, k+1)))

rango = list(range(1, k+1))

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    #print(name_column)
    # ahora saco los k mejores problemas para cada usuario
    df_users[name_column] = df_users.apply(lambda row: getKrecomFinal(row, i), axis=1)

# he incluido una columna para cada cada k recomendacion 
print(df_users)

      user_id                                     recommendation  \
11         18  [505, 509, 314, 122, 503, 109, 502, 310, 124, ...   
14         25  [130, 487, 308, 489, 247, 388, 155, 315, 465, ...   
39         60  [162, 10, 93, 119, 272, 70, 239, 437, 97, 17, ...   
90        130  [507, 122, 502, 23, 76, 316, 373, 322, 308, 26...   
241       414  [119, 256, 249, 507, 257, 159, 266, 88, 122, 5...   
255       443  [487, 308, 314, 373, 279, 322, 393, 489, 465, ...   
257       448  [247, 313, 487, 511, 231, 39, 239, 253, 62, 93...   
392       689  [88, 6, 509, 76, 65, 206, 203, 83, 365, 502, 7...   
512       912  [308, 342, 314, 257, 29, 258, 226, 57, 10, 440...   
523       935  [166, 358, 440, 88, 313, 206, 31, 383, 245, 76...   
933      1711  [486, 373, 31, 484, 383, 488, 440, 262, 322, 3...   
997      1893  [6, 446, 119, 51, 162, 507, 509, 440, 122, 502...   
1021     1952  [322, 209, 39, 191, 404, 437, 217, 222, 275, 2...   
1024     1955  [440, 322, 465, 404, 217, 88, 511

In [109]:
# elimino las columnas que no me interesan
del df_users['recommendation']

print(df_users)

      user_id k_recommendation_1 k_recommendation_2 k_recommendation_3  \
11         18              [505]         [505, 509]    [505, 509, 314]   
14         25              [130]         [130, 487]    [130, 487, 308]   
39         60              [162]          [162, 10]      [162, 10, 93]   
90        130              [507]         [507, 122]    [507, 122, 502]   
241       414              [119]         [119, 256]    [119, 256, 249]   
255       443              [487]         [487, 308]    [487, 308, 314]   
257       448              [247]         [247, 313]    [247, 313, 487]   
392       689               [88]            [88, 6]       [88, 6, 509]   
512       912              [308]         [308, 342]    [308, 342, 314]   
523       935              [166]         [166, 358]    [166, 358, 440]   
933      1711              [486]         [486, 373]     [486, 373, 31]   
997      1893                [6]           [6, 446]      [6, 446, 119]   
1021     1952              [322]      

In [110]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
4               [241, 128, 124, 256, 404, 469, 70, 86]       18
5                    [33, 29, 254, 308, 445, 253, 166]       25
8    [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...       60
13                       [254, 253, 308, 166, 49, 445]      130
22   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...      414
26             [134, 187, 256, 251, 90, 174, 147, 438]      443
27   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...      448
34                     [327, 23, 150, 349, 336, 6, 49]      689
44   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...      912
46          [95, 383, 262, 122, 44, 81, 252, 145, 336]      935
70   [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...     1711
73                       [471, 469, 355, 95, 243, 436]     1893
78         [374, 195, 109, 436, 95, 10, 446, 340, 379]     1952
80                            [44, 213, 136, 162, 243]     1955
90             [136, 253, 187, 342, 203,

In [111]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems}

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    list_recom_problems = list()
    list_recom_problems = df_users[name_column].tolist()
    
    name_column_metric = 'recom_problems_' + str(i)
    set_df_metric[name_column_metric] = list_recom_problems

# he generado un nuevo dataframe con todas las recomendaciones (desde 1 a k) y los problemas realizados en 
# el evaluation set para cada usuario al que quiero recomendar

metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems recom_problems_1  \
0              [241, 128, 124, 256, 404, 469, 70, 86]            [505]   
1                   [33, 29, 254, 308, 445, 253, 166]            [130]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...            [162]   
3                       [254, 253, 308, 166, 49, 445]            [507]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...            [119]   
5             [134, 187, 256, 251, 90, 174, 147, 438]            [487]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...            [247]   
7                     [327, 23, 150, 349, 336, 6, 49]             [88]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...            [308]   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]            [166]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...            [486]   
11                      [471, 469, 355, 95, 243, 436]              [6]   
12        [374, 195, 109, 436, 95, 10,

In [112]:
def one_hit(row, i):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [113]:
def mrr(row, i): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row[name_column])) and (encontrado == False):
            if row[name_column][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [114]:
def precision(row, i):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row[name_column]))

In [115]:
def recall(row, i):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [116]:
def f1(row, i):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    name_column_prec = 'precision_' + str(i)
    name_column_rec = 'recall_' + str(i)
    denominador = row[name_column_prec] + row[name_column_rec]
    
    if denominador == 0:
        return 0
    else:
        return (2 * row[name_column_prec] * row[name_column_rec]) / denominador

In [117]:
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    # ahora voy a calcular una metrica para cada usuario
    metric_df[name_one_hit] = metric_df.apply(lambda row: one_hit(row, i), axis=1)
    metric_df[name_mrr] = metric_df.apply(lambda row: mrr(row, i), axis=1)
    metric_df[name_precision] = metric_df.apply(lambda row: precision(row, i), axis=1)
    metric_df[name_recall] = metric_df.apply(lambda row: recall(row, i), axis=1)
    metric_df[name_f1] = metric_df.apply(lambda row: f1(row, i), axis=1)

print(metric_df)

                                        eval_problems recom_problems_1  \
0              [241, 128, 124, 256, 404, 469, 70, 86]            [505]   
1                   [33, 29, 254, 308, 445, 253, 166]            [130]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...            [162]   
3                       [254, 253, 308, 166, 49, 445]            [507]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...            [119]   
5             [134, 187, 256, 251, 90, 174, 147, 438]            [487]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...            [247]   
7                     [327, 23, 150, 349, 336, 6, 49]             [88]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...            [308]   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]            [166]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...            [486]   
11                      [471, 469, 355, 95, 243, 436]              [6]   
12        [374, 195, 109, 436, 95, 10,

In [118]:


f = open("C:/hlocal/TFM/vot_wpa", 'a')

# calculo la media de las metricas
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    result_one_hit = metric_df[name_one_hit].mean()
    result_precision = metric_df[name_precision].mean()
    result_mrr = metric_df[name_mrr].mean()
    result_recall = metric_df[name_recall].mean()
    result_f1 = metric_df[name_f1].mean()

    # lo muestro por consola
    
    print(i)
    print("###########")
    print("One hit ----------")
    print(result_one_hit)
    print("Precision ----------")
    print(result_precision)
    print("Mrr  ----------")
    print(result_mrr)
    print("Recall  ----------")
    print(result_recall)
    print("F1  ----------")
    print(result_f1)
    
    
    f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 

    
f.close()    
    

1
###########
One hit ----------
0.06153846153846154
Precision ----------
0.06153846153846154
Mrr  ----------
0.06153846153846154
Recall  ----------
0.006925533482910533
F1  ----------
0.012034739454094293
2
###########
One hit ----------
0.13846153846153847
Precision ----------
0.06923076923076923
Mrr  ----------
0.1
Recall  ----------
0.01568621724359429
F1  ----------
0.024719197562006925
3
###########
One hit ----------
0.23076923076923078
Precision ----------
0.08205128205128204
Mrr  ----------
0.13076923076923078
Recall  ----------
0.02864904676380086
F1  ----------
0.04052530802530802
4
###########
One hit ----------
0.3230769230769231
Precision ----------
0.08461538461538462
Mrr  ----------
0.15384615384615388
Recall  ----------
0.036877186145786396
F1  ----------
0.048898184110615646
5
###########
One hit ----------
0.4153846153846154
Precision ----------
0.09538461538461543
Mrr  ----------
0.1723076923076923
Recall  ----------
0.050470944566152635
F1  ----------
0.06235368088